## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read data csv

In [ ]:
data_file = "Stripe-82/stripe82_data.csv"
dat = pd.read_csv(data_file, dtype=object)
total = len(dat)

pz = (dat['photoZ1'] != '-9999') & (dat['photoZ2'] != '-9999')
dat = dat[pz]['objid'].astype({'objid':'string'})
valid = len(dat)

print("Total = {}".format(total))
print("Valid = {}".format(valid))

## Read band csv

In [ ]:
band_file = "Stripe-82/stripe82_pids.csv"
bdat = pd.read_csv(band_file, dtype=object)[['objid', 'bands']].astype({'objid':'string', 'bands':'string'})
obj_band = bdat[bdat['objid'].isin(dat)]

## Band-Count and Object-Bands Dicts

In [ ]:
band_stat = {b:[] for b in "ugri"}
objid_stat = {}

for _, row in obj_band.iterrows() :
    objid, bands = tuple((
        row[c] for c in ['objid', 'bands']
    ))
    for b in bands :
        band_stat[b].append(obj_band.loc[i, 'objid'])
        
    objid_stat[objid] = bands

## Band Correlation

In [ ]:
corrs = {}
for b1 in "ugri" :
    corr = {b:0 for b in "ugri" if b != b1}

    other_count = []
    single_band = 0
    for objid in band_stat[b1] :
        other = 0
        for b2 in objid_stat[objid] :
            if b1 != b2 :
                other += 1
                corr[b2] += 1

        if not other :
            single_band += 1
        else :
            other_count.append(other)

    uq, cnts = np.unique(np.array(other_count), return_counts=True)
    other_count = {uq[i]:cnts[i] for i in range(len(uq))}
    corrs[b1] = (corr, single_band, other_count)

# Overall Band Statistics

In [ ]:
max_corr = max([
    max([u for u,_ in corrs[b][2].items()]) 
    for b in "ugri"
])
print ("Maximum simultaneous detection of any object is in {} bands".format(max_corr+1))

max_band, max_count = '', 0
for b in "ugri" :
    if len(band_stat[b]) > max_count :
        max_count = len(band_stat[b])
        max_band = b
print ("Maximum detections occured in {}-band with {} detections".format(max_band, max_count))

## Individual Band Statitics

In [ ]:
for band in "ugri" :
    print ("INFO FOR {}-BAND".format(band))
    print ("Total detections in {}-band = {}".format(band, len(band_stat[band])))
    print ("Exclusive detections in {}-band".format(band, corrs[band][1]))
    print ("Band correlation with {}-band - ".format(band))
    for b, c in corrs[band][0].items() :
        print ("\t{}-band : {}".format(b, c))

    print ("Correlation count with {}-band - ".format(band))
    for u, c in corrs[b][2].items() :
        print ("\tNumber of {} other simultaneous detection(s) : {}".format(u, c))

    corr_max, corr_cnt = '', 0
    for b, c in corrs[band][0].items() :
        if c > corr_cnt :
            corr_max = b
            corr_cnt = c
    print ("Max correlation of {}-band is with {}-band carrying {} detections".format(band, corr_max, corr_cnt))
    
    if band != 'i' :
        print (65*"-")